- We are trying to translate a [sample] of the squad dataset into tamil and building one for us!

### blue-print...
- In the first cell, we do some house keeping stuff and setup precisely what we need.
- In the second cell, we are downloading the model which can translate to 12 different indic languages.

- Rest cells, I believe are intuitive :) (they include doing some prep stuff, laoding model and finally translating them)



# citation

**Give credits where it's due....**

```
@misc{ramesh2021samanantar,
      title={Samanantar: The Largest Publicly Available Parallel Corpora Collection for 11 Indic Languages},
      author={Gowtham Ramesh and Sumanth Doddapaneni and Aravinth Bheemaraj and Mayank Jobanputra and Raghavan AK and Ajitesh Sharma and Sujit Sahoo and Harshita Diddee and Mahalakshmi J and Divyanshu Kakwani and Navneet Kumar and Aswin Pradeep and Kumar Deepak and Vivek Raghavan and Anoop Kunchukuttan and Pratyush Kumar and Mitesh Shantadevi Khapra},
      year={2021},
      eprint={2104.05596},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}
```

Squad Dataset Used from Kaggle Datasets (by ananthu017) Ref -> https://www.kaggle.com/ananthu017/squad-csv-format

####

> - **PS Let me know if you are looking for a teammate!**
> - **If this helped, do let me know if there were improvements!"


# What you get as output from the kernel?
- Extra ~1k Tamil q&a machine translated for training! (Actually it's infinite because you can translate as many as you want)

# version notes

- in v3, Have updated the snips to remove newlines chars.

In [ ]:
# setting up the env... (will take some time, please be patient, require active internet connection)

# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./
!python setup.py build_ext --inplace
%cd ..

# this import might not work without restarting runtime
# from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# english-to-indic tranlated (machine generated dataset)
# model from https://indicnlp.ai4bharat.org/indic-trans/
# will take some time, so sit back and sip your coffee! (please avoid downloading again and again, push it to a kaggle dataset :pray:)

!wget https://storage.googleapis.com/samanantar-public/V0.2/models/en-indic.zip
!unzip ./en-indic.zip
!rm -f en-indic.zip

In [ ]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
from mosestokenizer import *
from indicnlp.tokenize import sentence_tokenize

# this import might not work without restarting runtime (just restart (clear outputs) and then execute this cell only)
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

%cd ./indicTrans
# load the tranlation model from that directory
# from indicTrans.inference.engine import Model # because of this import, we have to do cd...
# en2indic_model = Model(expdir='/kaggle/working/en-indic')
# en2indic_model

In [ ]:
# let's load the sample squad dataset....
import pandas as pd

df = pd.read_csv(
    "/kaggle/input/squad-csv-format/SQuAD_csv.csv", 
    usecols=["context", "question", "id", "text", "answer_start"],
    nrows = 2**10, # sample
)
df.drop_duplicates(subset=["context"], inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.shape)

# items to translate
en_context = list(df["context"].values)
en_question = list(df["question"].values)
en_answer_text = list(df["text"].values)

df.head()

In [ ]:
# writing the paragraphs line-by-line

import io
from tqdm.auto import tqdm

with io.open("en_paragraphs.txt", "w") as f_ptr:
    for line in en_context:
        f_ptr.write(line.replace("\n", "").strip())
        f_ptr.write("\n")

with io.open("en_question.txt", "w") as f_ptr:
    for line in en_question:
        f_ptr.write(line.replace("\n", "").strip())
        f_ptr.write("\n")

with io.open("en_answer_text.txt", "w") as f_ptr:
    for line in en_answer_text:
        f_ptr.write(line.strip())
        f_ptr.write("\n")

In [ ]:
# getting the line counts
!wc -l "en_paragraphs.txt" "en_answer_text.txt" "en_question.txt"

In [ ]:
%%writefile joint_translate_updated.sh

#!/bin/bash
echo `date`
infname=$1
outfname=$2
src_lang=$3
tgt_lang=$4
exp_dir=$5
ref_fname=$6

SRC_PREFIX='SRC'
TGT_PREFIX='TGT'

#`dirname $0`/env.sh
SUBWORD_NMT_DIR='subword-nmt'
model_dir=$exp_dir/model
data_bin_dir=$exp_dir/final_bin

### normalization and script conversion

echo "Applying normalization and script conversion"
input_size=`python scripts/preprocess_translate.py $infname $outfname.norm $src_lang true`
echo "Number of sentences in input: $input_size"

### apply BPE to input file

echo "Applying BPE"
python $SUBWORD_NMT_DIR/subword_nmt/apply_bpe.py \
    -c $exp_dir/vocab/bpe_codes.32k.${SRC_PREFIX} \
    --vocabulary $exp_dir/vocab/vocab.$SRC_PREFIX \
    --vocabulary-threshold 5 \
    < $outfname.norm \
    > $outfname._bpe

# not needed for joint training
# echo "Adding language tags"
python scripts/add_tags_translate.py $outfname._bpe $outfname.bpe $src_lang $tgt_lang

### run decoder

echo "Decoding"

src_input_bpe_fname=$outfname.bpe
tgt_output_fname=$outfname

fairseq-interactive  $data_bin_dir \
    -s $SRC_PREFIX -t $TGT_PREFIX \
    --distributed-world-size 1 --fp16 \
    --path $model_dir/checkpoint_best.pt \
    --batch-size 32 --buffer-size 1024 --data-buffer-size 8 --beam 5 --remove-bpe \
    --skip-invalid-size-inputs-valid-test \
    --user-dir model_configs \
    --input $src_input_bpe_fname  >  $tgt_output_fname.log 2>&1


echo "Extracting translations, script conversion and detokenization"
# this part reverses the transliteration from devnagiri script to target lang and then detokenizes it.
python scripts/postprocess_translate.py $tgt_output_fname.log $tgt_output_fname $input_size $tgt_lang true
echo "Translation completed"

In [ ]:
%%time

# joint_translate.sh is a shell script that takes args in the following order as inputs <src_file>, <output_fname>, <src_lang>, <tgt_lang>, <model_folder>
# it's quite fast if you are using GPU, do increase the. batch size from 64 as well in the shell snip that's hard-coded..
# <6 mins for ~90k sentences is not bad at all....

# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
# model_folder -> where's the model file located


!chmod 777 ./joint_translate_updated.sh

!./joint_translate_updated.sh en_paragraphs.txt ta_paragraphs.txt "en" "ta" '../en-indic'
!./joint_translate_updated.sh en_question.txt ta_question.txt "en" "ta" '../en-indic'
!./joint_translate_updated.sh en_answer_text.txt ta_answer_text.txt "en" "ta" '../en-indic'

In [ ]:
%%time

# en_context, en_question, en_answer_text -- for english stuff

# read the context from the file "ta_paragraphs.txt"
with io.open("ta_paragraphs.txt", "r") as f_ptr:
    ta_context = [line.strip() for line in f_ptr]

# read the context from the file "ta_question.txt"
with io.open("ta_question.txt", "r") as f_ptr:
    ta_question = [line.strip() for line in f_ptr]

# read the context from the file "ta_answer_text.txt"
with io.open("ta_answer_text.txt", "r") as f_ptr:
    ta_en_answer_text = [line.strip() for line in f_ptr]

In [ ]:
cnt = 0
good_ids = set()
for idx in range(len(ta_en_answer_text)):
    if ta_en_answer_text[idx] in ta_context[idx]:
        # basically naswer is present as-is in the translated text
        cnt += 1
        good_ids.add(idx)
cnt / len(ta_en_answer_text), len(good_ids) # 22% is not bad!

In [ ]:
import pickle

with open("ta_question.pkl", "wb") as f_ptr:
    pickle.dump(ta_question, f_ptr)

with open("ta_en_answer_text.pkl", "wb") as f_ptr:
    pickle.dump(ta_en_answer_text, f_ptr)

In [ ]:
! cp "ta_paragraphs.txt" "/kaggle/working"
! cp "ta_question.txt" "/kaggle/working"
! cp "ta_answer_text.txt" "/kaggle/working"

! cp "en_paragraphs.txt" "/kaggle/working"
! cp "en_question.txt" "/kaggle/working"
! cp "en_answer_text.txt" "/kaggle/working"

! cp "ta_en_answer_text.pkl" "/kaggle/working"
! cp "ta_question.pkl" "/kaggle/working"

In [ ]:
# prepare the dataframe that we can use directly.
# sample data
ta_question[0], ta_en_answer_text[0], ta_context[0], en_context[0], en_answer_text[0], en_question[0]

In [ ]:
from dataclasses import dataclass

@dataclass
class TamilExtraData:
    ta_context :str
    ta_question :str
    ta_answer_text :str
    en_context :str
    en_question :str
    en_answer_text :str

In [ ]:
final_data = []
for idx in good_ids:
    final_data.append(
        TamilExtraData(
            ta_context=ta_context[idx], ta_answer_text=ta_en_answer_text[idx], ta_question=ta_question[idx],
            en_context=en_context[idx], en_answer_text=en_answer_text[idx], en_question=en_question[idx],
        ).__dict__
    )

In [ ]:
df = pd.DataFrame.from_records(final_data)
df["answer_start"] = df[["ta_context", "ta_answer_text"]].apply(lambda row: row[0].find(row[1]), axis=1)
# df["answer_end"] = df[["context", "answer_text"]].apply(lambda row: row[0].find(row[1]) + len(row[1]), axis=1)
df

In [ ]:
df.to_csv("/kaggle/working/tamil_extra_translated_data.csv", index=None) # almost extra 1000 tamil dataset rows... Enjoy People...

In [ ]:
# clean-up cell [be SUPER careful when you run this guy]
!rm -rf *
%cd ..
!rm -rf ./en-indic/
!rm -rf ./fairseq/
!rm -rf ./indicTrans/
!ls

In [ ]:
!ls # much better!

In [ ]:
# translated texts...
df